We have a few ETL functions from our Python EDA that we want to migrate to, for example, a cloud-based setting.

Notable ETL functions include:

- Placing the data in space and time -- northeastern US, starting on 2004,3,23
    - by locating thanksgiving and christmas we could deduce a seasonal 

    - arimax on the sales section level?

    

In [1]:
from database import engine, get_db
from my_url import _SQLALCHEMY_DATABASE_URL
from dunnhumby_data_warehouse import DunnHumbyDataWarehouse
import pandas as pd 
a = DunnHumbyDataWarehouse()

2023-11-03 16:55:54,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-03 16:55:54,388 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
df = pd.read_csv('../data/daily_campaign_sales.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/daily_campaign_sales.csv'

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:

df.to_sql(name='daily_campaign_sales',
                con=_SQLALCHEMY_DATABASE_URL,
                #  if_exists='append',
                if_exists='replace',
                chunksize=999,
                method='multi')

17597

In [ ]:
a.get_table('daily_campaign_sales')

,index,DAY,SALES_VALUE,campaign_id
0,0,101,63.77,10
1,1,102,93.07,10
2,2,103,89.46,10
3,3,104,49.88,10
4,4,105,26.88,10
...,...,...,...,...
17592,17592,704,164.03,9
17593,17593,705,272.35,9
17594,17594,706,192.88,9
17595,17595,707,131.86,9


In [ ]:
# startup the API server using `uvicorn main:app --reload` in the shell

# Derived Tables

The code below is being adapted from previous work I did on DunnHumby. 

Because of the data coming out of the db now, some column names need to be lowercased.

The operations below represent a complex combination of data cleaning steps and table joins; there's way too much going on

In [ ]:
# Transforming TRANSACTION DATA AND PRODUCT


import datetime as dt
import pandas as pd
import numpy as np



# def hh_summary_create():
# TODO:  ### consider dropping unnecessary columns
### this currently rips directly from the data source, making the data lake obsolete
### 


### TRANSACTION DATA

transactions = a.get_table('transaction_data')
#pd.read_csv('data/transaction_data.csv')
#self.get_table('transaction_data')

transactions = transactions.drop('index',axis=1)

# add datetime values. this should be back-integrated with the data ingestion procedure, now that we have it
day1 = dt.datetime(2004, 3, 23) # as derived in transactions notebook; datetime for 'DAY' == 1
ineedthismany = transactions['DAY'].max()
last = day1 + dt.timedelta(days=int(ineedthismany))
date_range = pd.date_range(day1, last) # date range for our data
# map datetime index to DAY; enumerate() indexes from 0, so we add 1
date_map = {i+1:x for i, x in enumerate(date_range)}
# truncate data

output = transactions['TRANS_TIME'].astype(str).str.zfill(4)
# split to hours and minutes
hours = output.str[:2]
minutes = output.str[2:]

# convert to timedelta
hours = pd.to_timedelta(hours.astype('int'), unit='hour')
minutes = pd.to_timedelta(minutes.astype('int'), unit='minute')
time_of_transaction = hours + minutes

# add date and time
transactions['datetime'] = transactions['DAY'].map(date_map) + time_of_transaction



### todo: remove unnecessary columns? --> DAY is used by streamlit_front/campaign_performance, etc.
#transactions.drop(['DAY', 'TRANS_TIME'], axis=1)

# filtering empty sales rows..
transactions = transactions.loc[(transactions['QUANTITY'] > 0) & 
                (transactions['SALES_VALUE'] > 0)]


# filtering on datetime (data acquisition inconsistency?)
transactions = transactions[(transactions['datetime'] >= "2004-7-1") &
                (transactions['datetime'] < "2006-3-1")]




###  PRODUCT 
prod = a.get_table('product')
#pd.read_csv('../data/product.csv')
prod.drop('index', axis=1, inplace=True)

merged = transactions.merge(prod.drop('CURR_SIZE_OF_PRODUCT', axis=1), on='PRODUCT_ID')


# Remove Gasoline Sales
merged.drop(merged[merged['SUB_COMMODITY_DESC']=='GASOLINE-REG UNLEADED'].index, axis=0, inplace=True)
merged.drop(merged[merged['COMMODITY_DESC']=='GASOLINE-REG UNLEADED'].index, axis=0, inplace=True)

# Add section labels
with open('Section_Labels.txt', 'r') as f:
    product_section_map = eval(f.readlines()[0])
ser = merged['COMMODITY_DESC'].map(product_section_map) # hardcoded;
ser = ser.fillna('misc') # for exceptions ?

merged['Section Labels'] = ser

# Remove one-day transactions -- outliers?
def one_day_transactions(df) -> list:
    no_days = df.groupby('household_key').agg({'DAY':'nunique'})
    return list(no_days[no_days['DAY'] == 1].index)

# remove households with only 1 day of purchases;
merged = merged[~merged['household_key'].isin(one_day_transactions(merged))]    



In [65]:
merged

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,datetime,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,Section Labels
0,2319,28179463886,101,909714,1,0.79,427,0.00,119,15,0.0,0.0,2004-07-01 01:19:00,2,PRODUCE,National,APPLES,APPLES GRANNY SMITH (BULK&BAG),produce
1,243,28197355878,102,909714,1,0.72,323,0.00,1011,15,0.0,0.0,2004-07-02 10:11:00,2,PRODUCE,National,APPLES,APPLES GRANNY SMITH (BULK&BAG),produce
2,707,28210545743,103,909714,1,2.35,429,0.00,1119,15,0.0,0.0,2004-07-03 11:19:00,2,PRODUCE,National,APPLES,APPLES GRANNY SMITH (BULK&BAG),produce
3,329,28210628714,103,909714,1,2.10,450,0.00,2037,15,0.0,0.0,2004-07-03 20:37:00,2,PRODUCE,National,APPLES,APPLES GRANNY SMITH (BULK&BAG),produce
4,1029,28210676830,103,909714,1,0.81,329,0.00,1148,15,0.0,0.0,2004-07-03 11:48:00,2,PRODUCE,National,APPLES,APPLES GRANNY SMITH (BULK&BAG),produce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404767,2384,42250952449,708,18148536,2,1.76,389,0.00,1751,102,0.0,0.0,2006-02-28 17:51:00,69,GROCERY,Private,COLD CEREAL,ALL FAMILY CEREAL,grain_goods
2404768,2347,42250975392,708,14077733,1,2.50,438,-0.79,1054,102,0.0,0.0,2006-02-28 10:54:00,69,MEAT-PCKGD,Private,LUNCHMEAT,PEPPERONI/SALAMI,meat
2404769,2280,42250975829,708,18106248,1,1.50,438,0.00,1307,102,0.0,0.0,2006-02-28 13:07:00,1687,DRUG GM,National,EASTER,EASTER FILL EGGS,seasonal
2404770,886,42250982055,708,5589058,3,1.50,402,0.00,1910,102,0.0,0.0,2006-02-28 19:10:00,5423,MEAT-PCKGD,National,LUNCHMEAT,MISCELLANEOUS,meat


In [ ]:

# write data using pandas (for now)
# merged.to_sql(name='_merged_no_hh',
#             con=_SQLALCHEMY_DATABASE_URL,
#             # if_exists='append',
#            if_exists='replace',
#             chunksize=999,
#             method='multi')
# HouseHold Analytics Pivot

### Creating Customer-Level Sales by Section Labels
section_dummies = pd.get_dummies(merged['Section Labels'])
#print(section_dummies.sum().sum())
# the transaction-level (item) binary flags for
#section_dummies.sum()

### total sales by section, for each household
### may want to break this out further, into ie. weekly, bi-weekly, monthly, or quarterly sales totals for each section
section_sales = section_dummies.apply(lambda x : x * merged['SALES_VALUE'])
daily_hh_spend_by_section = section_sales.join(merged[['household_key', 'DAY',]])
daily_hh_spend_by_section

daily_hh_spend_by_section =daily_hh_spend_by_section.groupby(['DAY','household_key', ]).agg({col:'sum' for col in section_dummies.columns})

daily_hh_spend_by_section
daily_hh_spend_by_section['total_sales'] = daily_hh_spend_by_section.sum(axis=1)

,alcohol,beverages,concessions,dairy,drug,garden,grain_goods,grocery,home_family,junk_food,kitchen,meat,misc,produce,seasonal,household_key,DAY
0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.79,0.0,2319,101
1,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.72,0.0,243,102
2,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.35,0.0,707,103
3,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.10,0.0,329,103
4,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.81,0.0,1029,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404767,0.0,0.0,0.0,0.0,0.0,0.00,1.76,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2384,708
2404768,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,2.5,0.0,0.00,0.0,2347,708
2404769,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.5,2280,708
2404770,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.5,0.0,0.00,0.0,886,708


In [96]:
# data output

#daily_hh_spend_by_section.to_csv('data/daily_hh_spend_by_section.csv')


# daily_hh_spend_by_section.to_sql(name='daily_hh_spend',
#             con=_SQLALCHEMY_DATABASE_URL,
#             # if_exists='append',
#            if_exists='replace',
#             chunksize=999,
#             method='multi')

198328

In [1]:
import requests



In [4]:
res = requests.get(f"http://localhost:8000/hh/1")
data = res.json()
data

{'index': 0,
 'household_key': 1,
 'age_45_plus': True,
 'income_50K_plus': False,
 'single_couple_family': 2,
 'has_kids': False,
 'single': False,
 'couple': True,
 'r_score': 5,
 'f_score': 3,
 'm_score': 4,
 'rfm_score': 12,
 'rfm_bins': 3,
 'alcohol': 6.48,
 'beverages': 314.83,
 'concessions': 20.33,
 'dairy': 362.76,
 'drug': 53.440000000000005,
 'garden': 0.0,
 'grain_goods': 668.41,
 'grocery': 230.24,
 'home_family': 537.29,
 'junk_food': 764.02,
 'kitchen': 1.59,
 'meat': 379.04,
 'misc': 303.82,
 'produce': 477.82,
 'seasonal': 0.0,
 'total_sales': 4120.07}

In [ ]:

# ADDING RFM SCORE

last_days = merged.groupby('household_key')['DAY'].max()
R = pd.cut(last_days, [0, 525, 615, 675, 700, np.inf], labels=[1,2,3,4,5], ordered=True).astype('int')
num_baskets = merged.groupby('household_key')['BASKET_ID'].nunique()
grouper = merged.groupby('household_key')['DAY']
days_in_data = grouper.max() - grouper.min() + 1 #(no day 0 in our data)
transactions_per_day = num_baskets/days_in_data
F = pd.qcut(transactions_per_day, 5, labels=[1,2,3,4,5]).astype('int')
M = pd.qcut(np.log(merged.groupby('household_key')['SALES_VALUE'].sum()), 5, labels=[1,2,3,4,5]).astype('int')
temp = pd.concat([R, F, M], axis=1)
temp.columns = ['r_score', 'f_score', 'm_score']
temp['rfm_score'] = temp.sum(axis=1).astype(int)
customer_ranks = pd.cut(temp['rfm_score'], bins=[0,6,9,13,15, np.inf], labels=[1,2,3,4,5], right=False).astype(int)
customer_ranks = pd.DataFrame(customer_ranks) ## THIS IS HACKY BRO
temp['rfm_bins'] = customer_ranks

hh_agg = temp.merge(hh_agg, on='household_key').reset_index()





# DEMOGRAPHIC DATA



demo = a.get_table('hh_demographic')

#pd.read_csv('../data/hh_demographic.csv')
## Alternate Mappings
demo['age_45_plus'] = demo['AGE_DESC'].map({ '19-24':0,
                                        '25-34':0,
                                        "35-44":0,
                                        '45-54':1,
                                        '55-64':1,
                                        '65+':1,
                                        })


demo['income_50K_plus'] = demo['INCOME_DESC'].map({
                        'Under 15K': 0,
                            '15-24K': 0,
                            '25-34K': 0,
                            '35-49K': 0,
                            '50-74K': 1,
                            '75-99K': 1,
                            '100-124K': 1,
                            '125-149K': 1,
                            '150-174K': 1,                   
                            '175-199K': 1,  
                            '200-249K': 1,
                            '250K+': 1,
                        })

# leaving household_size desc IN as a category; single, couple, 3+
demo['single_couple_family'] = demo['HOUSEHOLD_SIZE_DESC'].map({'1':1, '2':2, '3':3,'4':3,'5+':3})
#demo['single_couple_family'] = pd.Categorical(demo['single_couple_family'], 
#                                                [1,2,3,])


demo['has_kids'] = np.where((demo['HH_COMP_DESC'] == '1 Adult Kids') |
                            (demo['HH_COMP_DESC'] == '2 Adults Kids') |
                            (demo['HOUSEHOLD_SIZE_DESC'].isin(['3', '4', '5+'])),
                            1, 0)

demo['single'] =  np.where((demo['HH_COMP_DESC'] == 'Single Female') |
                            (demo['HH_COMP_DESC'] == 'Single Male') |
                            (demo['HOUSEHOLD_SIZE_DESC'] == '1'),
                            1, 0)

demo['couple'] =  np.where((demo['HH_COMP_DESC'] == '2 Adults No Kids'),
                            1, 0)

demo = demo.drop(['AGE_DESC', 'MARITAL_STATUS_CODE', 
            'INCOME_DESC', 'HOMEOWNER_DESC', 'HH_COMP_DESC', 
            'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC'], axis=1)


# binary flags for low and high income; low and high age; based on distributions of values in a known data set?

hh_summary = demo.merge(hh_agg, on='household_key')

hh_summary
### HH_SUMMARY OUTPUT
# hh_summary.set_index('index', inplace=True)
# hh_summary.to_sql(name='hh_summary',
#                 con=_SQLALCHEMY_DATABASE_URL,
#             #      if_exists='append',
#                 if_exists='replace',
#                 #chunksize=999,
#                 method='multi')

# hh_summary.to_csv('data/hh_summary.csv')

# Streamlit App Data Reconstruction

The streamlit apps I constructed previously depend on the result of various ETLs which were a part of my prior data processing workflow (the DTCJ python "package").

Creating a package of python functions which transform raw data into tangible insights was a great learning process over winter 2021/spring 2022; but the functionality I was looking for is much better served in a persisted database.

In the following cells I'll be simply porting the constructed data into the database; if it were necessary, I would create the data pipeline process once again.

I realize that this data warehouse is sort of patched together; but this has been yet another great learning experience for me. Getting hands on experience defining my own database schemas and API transaction models to power a front-end delivery is a completion of sorts in my data journey. 

While I still have a long ways to go in terms of implementing MLOps or DevOps procedures, I stand by my original assessment that basic analytics can answer many business questions -- and that to have reliable insights, one must ensure the quality of the data first.

In [ ]:
import pandas as pd
import glob

In [ ]:

# THIS FUNCTION HAS BEEN ALTERED
def make_date_map(df, dates='DAY'):
    # 'DAY' 1 == 2004-03-23
    day1 = datetime.datetime(2004, 3, 23) # as derived in transactions notebook; datetime for 'DAY' == 1
    #ineedthismany = df[dates].max()
    last = day1 + datetime.timedelta(days=int(999))
    date_range = pd.date_range(day1, last) # date range for our data
    # map datetime index to DAY; enumerate() indexes from 0, so we add 1
    date_map = {i+1:x for i, x in enumerate(date_range)}

    #output = df[dates].map(date_map)
    #output = pd.to_datetime(output)
    return date_map

In [ ]:
import datetime
date_map = make_date_map(df, dates='First Day')
#make_date_map(df, dates='Last Day')

In [ ]:

# this table holds both a list of products, for each campaign; 
# it's unstructured data
df = pd.read_csv('streamlit_frontends/campaign_performance/data/campaign_summary.csv', index_col=0)
df.reset_index(names='campaign_id', inplace=True)
df.head(2)

,campaign_id,First Day,Last Day,Duration,Listed Products,Section Label Counts,Listed Products Total Sales,Listed Products Sales Before,Listed Products Sales During,Listed Products Sales After,avg. total,avg. before,avg. during,avg. after,first,last
0,1,346,383,38,"[28929, 29096, 32387, 32805, 33198, 34180, 343...","{'dairy': 13496, 'produce': 7565, 'beverages':...",56764.87,21854.53,4145.12,30765.22,92.904861,88.839553,109.082105,93.511307,346.0,383.0
1,2,351,383,33,"[49910, 61481, 61509, 67573, 80730, 82937, 857...","{'produce': 5763, 'grocery': 2952, 'junk_food'...",47328.04,19708.03,2713.86,24906.15,77.459967,78.518048,82.238182,75.702584,351.0,383.0


In [ ]:
df['campaign_start_date'] = df['First Day'].map(date_map)

df['campaign_end_date'] = df['First Day'].map(date_map)

In [ ]:
df.to_csv('../data/campaign_summary.csv')

In [ ]:
#df = pd.read_csv('streamlit_frontends/household_analytics_table/data/hh_summary.csv')
# df1 = pd.read_csv()
    
# this table holds the total sales values for the products from each campaign; before, during, and after the campaign
df1 = pd.read_csv('streamlit_frontends/campaign_performance/data/campaign_sales.csv', index_col=0)
# df1

In [ ]:
df1

,total,before,during,after,first,last
1,93.445925,88.839553,109.082105,94.516140,346.0,383.0
2,78.007447,78.518048,82.238182,76.719331,351.0,383.0
3,70.839051,70.327734,77.519123,69.533900,356.0,412.0
4,57.659460,58.944632,56.312727,56.294383,372.0,404.0
5,48.305499,55.889928,52.226000,40.548970,377.0,411.0
6,3.272602,1.095358,4.718788,5.306272,393.0,425.0
7,23.316039,21.576309,21.755429,25.196143,398.0,432.0
8,4094.791391,4027.016442,4261.931020,4113.705516,412.0,460.0
9,175.363142,163.660060,183.994848,188.771143,435.0,467.0
10,62.125843,67.206006,53.849394,54.313733,463.0,495.0


In [ ]:

# proportion of sales by section; by campaign
# df2 = pd.read_csv('streamlit_frontends/campaign_performance/data/section_value.csv', index_col=0)
# #df2 = df2.T.reset_index(names='campaign_id')#
# df2 = df2.drop('sum', axis=0)
# df2


In [ ]:
## aggregate campaign-specific daily sales data

output = []
for x in glob.glob('streamlit_frontends/campaign_performance/data/camp_sales/*'):
    temp = pd.read_csv(x)
    camp_id = x.split('/')[-1].split('\\')[-1].split('_')[-2]
    temp['campaign_id'] = camp_id
    output.append(temp)

output = pd.concat(output)


In [ ]:
# daily campaign sales OUT
output.to_csv('../data/daily_campaign_sales.csv')

In [ ]:
output

,DAY,SALES_VALUE,campaign_id
0,101,63.77,10
1,102,93.07,10
2,103,89.46,10
3,104,49.88,10
4,105,26.88,10
...,...,...,...
602,704,164.03,9
603,705,272.35,9
604,706,192.88,9
605,707,131.86,9
